In [ ]:
!pip install transformers==4.47.1 numpy==1.26.4 scipy==1.13.1 scikit-learn==1.6.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 10.1 MB 11.3 MB/s 
     |████████████████████████████████| 18.2 MB 12.6 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.2
    Uninstalling transformers-4.48.2:
      Successfully uninstalled transformers-4.48.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW, get_scheduler
from tqdm import tqdm
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.model_selection import train_test_split


In [ ]:
train_path = "./train.csv"
test_path = "./test.csv"
train = pd.read_csv(train_path, encoding='utf-8-sig')
test = pd.read_csv(test_path, encoding='utf-8-sig')

In [ ]:
train.head()

,ID,input,output
0,TRAIN_00000,별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈...,별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자...
1,TRAIN_00001,잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ,잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ
2,TRAIN_00002,절테 간면 않 된는 굣 멥몫,절대 가면 안 되는 곳 메모
3,TRAIN_00003,야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥류만 묵...,아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...
4,TRAIN_00004,집윈 축쳐눌료 딴너왓눈뎁 카셩뷔 좋곱 칼쿰한네올. 쩌럼한뒈 뮬콰 욺료토 잊쿄 빻토 ...,지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...


데이터 증강

In [ ]:
def decompose_hangul(char):

    base_code = ord(char) - 0xAC00
    initial_idx = base_code // 588
    medial_idx = (base_code % 588) // 28
    final_idx = base_code % 28

    initial = chr(initial_idx + 0x1100)  # 초성
    medial = chr(medial_idx + 0x1161)    # 중성
    final = chr(final_idx + 0x11A7) if final_idx > 0 else ''  # 종성

    return initial, medial, final

In [ ]:
import random

# 초성 변형 규칙 (거센소리, 된소리 변형) - 초성 코드 사용
initial_transform = {
    chr(0x1100): [chr(0x1101), chr(0x110F), chr(0x1100)],  # ㄱ -> ㄲ,ㅋ,ㄱ
    chr(0x1101): [chr(0x1100), chr(0x110F), chr(0x1101)],  # ㄲ -> ㄱ,ㅋ,ㄲ
    chr(0x1102): [chr(0x1102)],  # ㄴ
    chr(0x1103): [chr(0x1104), chr(0x1110), chr(0x1103)],  # ㄷ -> ㄸ,ㅌ,ㄷ
    chr(0x1104): [chr(0x1103), chr(0x1110), chr(0x1104)],  # ㄸ -> ㄷ,ㅌ,ㄸ
    chr(0x1105): [chr(0x1105)],  # ㄹ
    chr(0x1106): [chr(0x1106)],  # ㅁ
    chr(0x1107): [chr(0x1108), chr(0x1111), chr(0x1107)],  # ㅂ -> ㅃ,ㅍ,ㅂ
    chr(0x1108): [chr(0x1107), chr(0x1111), chr(0x1108)],  # ㅃ -> ㅂ,ㅍ,ㅃ
    chr(0x1109): [chr(0x110A), chr(0x1109)],  # ㅅ -> ㅆ,ㅅ
    chr(0x110A): [chr(0x1109), chr(0x110A)],  # ㅆ -> ㅅ,ㅆ
    chr(0x110B): [chr(0x110B)],  # ㅇ
    chr(0x110C): [chr(0x110D), chr(0x110E), chr(0x110C)],  # ㅈ -> ㅉ,ㅊ,ㅈ
    chr(0x110D): [chr(0x110C), chr(0x110E), chr(0x110D)],  # ㅉ -> ㅈ,ㅊ,ㅉ
    chr(0x110E): [chr(0x110C), chr(0x110D), chr(0x110E)],  # ㅊ -> ㅈ,ㅉ,ㅊ
    chr(0x110F): [chr(0x1100), chr(0x1101), chr(0x110F)],  # ㅋ -> ㄱ,ㄲ,ㅋ
    chr(0x1110): [chr(0x1103), chr(0x1104), chr(0x1110)],  # ㅌ -> ㄷ,ㄸ,ㅌ
    chr(0x1111): [chr(0x1107), chr(0x1108), chr(0x1111)],  # ㅍ -> ㅂ,ㅃ,ㅍ
    chr(0x1112): [chr(0x1112)]   # ㅎ
}

# 중성 변형 규칙 (발음 비슷) - 중성 코드 사용
medial_transform = {
    chr(0x1161): [chr(0x1163), chr(0x116A), chr(0x1161)],     # ㅏ -> ㅑ,ㅘ,ㅏ
    chr(0x1165): [chr(0x1167), chr(0x116F), chr(0x1165)],     # ㅓ -> ㅕ,ㅝ,ㅓ
    chr(0x1169): [chr(0x1169), chr(0x116D)],     # ㅗ -> ㅗ,ㅛ,
    chr(0x116E): [chr(0x116E), chr(0x1172)],     # ㅜ -> ㅜ,ㅠ,
    chr(0x1173): [chr(0x1174), chr(0x1173)],                  # ㅡ -> ㅢ,ㅡ
    chr(0x1175): [chr(0x1171), chr(0x1174), chr(0x1175)],     # ㅣ -> ㅟ,ㅢ,ㅣ
    chr(0x1162): [chr(0x1162), chr(0x1164), chr(0x1166), chr(0x1168), chr(0x116C), chr(0x116B), chr(0x1170)],  # ㅐ -> ㅐ,ㅒ,ㅔ,ㅖ,ㅚ,ㅙ,ㅞ
    chr(0x1166): [chr(0x1162), chr(0x1164), chr(0x1166), chr(0x1168), chr(0x116C), chr(0x116B), chr(0x1170)],  # ㅔ -> ㅐ,ㅒ,ㅔ,ㅖ,ㅚ,ㅙ,ㅞ
    chr(0x116B): [chr(0x1162), chr(0x1164), chr(0x1166), chr(0x1168), chr(0x116C), chr(0x116B), chr(0x1170)],  # ㅙ -> ㅐ,ㅒ,ㅔ,ㅖ,ㅚ,ㅙ,ㅞ
    chr(0x116C): [chr(0x1162), chr(0x1164), chr(0x1166), chr(0x1168), chr(0x116C), chr(0x116B), chr(0x1170)],  # ㅚ -> ㅐ,ㅒ,ㅔ,ㅖ,ㅚ,ㅙ,ㅞ
    chr(0x116F): [chr(0x1167), chr(0x1165)],                  # ㅝ -> ㅕ,ㅓ
    chr(0x1168): [chr(0x1162), chr(0x1164), chr(0x1166), chr(0x1168), chr(0x116C), chr(0x116B), chr(0x1170)],  # ㅖ -> ㅐ,ㅒ,ㅔ,ㅖ,ㅚ,ㅙ,ㅞ
    chr(0x1164): [chr(0x1162), chr(0x1164), chr(0x1166), chr(0x1168), chr(0x116C), chr(0x116B), chr(0x1170)],  # ㅒ -> ㅐ,ㅒ,ㅔ,ㅖ,ㅚ,ㅙ,ㅞ
    chr(0x116A): [chr(0x116B), chr(0x116C), chr(0x1170)],    # ㅘ -> ㅑ,ㅘ,ㅏ
    chr(0x1171): [chr(0x1171), chr(0x1175), chr(0x1174)],                  # ㅟ -> ㅟ,ㅢ,ㅣ
    chr(0x1174): [chr(0x1173), chr(0x1174), chr(0x1175), chr(0x1171)],    # ㅢ -> ㅡ,ㅣ,ㅢ,ㅟ
    chr(0x116B): [chr(0x1162), chr(0x1164), chr(0x1166), chr(0x1168), chr(0x116C), chr(0x116B), chr(0x1170)],  # ㅙ -> ㅐ,ㅒ,ㅔ,ㅖ,ㅚ,ㅙ,ㅞ
    chr(0x1170): [chr(0x1162), chr(0x1164), chr(0x1166), chr(0x1168), chr(0x116C), chr(0x116B), chr(0x1170)],  # ㅞ -> ㅐ,ㅒ,ㅔ,ㅖ,ㅚ,ㅙ,ㅞ
    chr(0x116D): [chr(0x116D), chr(0x1169)],                               # ㅛ -> ㅗ, ㅛ
    chr(0x1172): [chr(0x116E), chr(0x1172)],                               # ㅠ -> ㅜ, ㅠ
    chr(0x1167): [chr(0x1165), chr(0x1167)]                                # ㅕ -> ㅓ, ㅕ
}
# 종성 코드 리스트
final_consonants = [chr(0x11A8 + i) for i in range(28)]  # 종성 코드 (0x11A8-0x11C2)


def transform_initial(initial):
    if initial in initial_transform:
        return random.choice(initial_transform[initial])
    return initial

def transform_medial(medial):
    if medial in medial_transform:
        return random.choice(medial_transform[medial])
    return medial

def transform_final(final):
    if final:  # 종성이 있을 경우
        return random.choice(final_consonants)
    return ''  # 종성이 없으면 빈 문자열 반환

In [ ]:
def compose_hangul(decomposed):
    HANGUL_BASE = 0xAC00
    INITIAL_BASE = 0x1100
    MEDIAL_BASE = 0x1161
    FINAL_BASE = 0x11A7

    initial, medial, final = decomposed
    initial_idx = ord(initial) - INITIAL_BASE
    medial_idx = ord(medial) - MEDIAL_BASE
    final_idx = ord(final) - FINAL_BASE if final else 0

    code = HANGUL_BASE + (initial_idx * 588) + (medial_idx * 28) + final_idx
    return chr(code)


In [ ]:
def transform_text(text):
    transformed_text = []
    for char in text:
        if '가' <= char <= '힣':  # 한글 음절인 경우만 처리
            initial, medial, final = decompose_hangul(char)
            # 초성, 중성, 종성 변형
            new_initial = transform_initial(initial)
            new_medial = transform_medial(medial)
            new_final = transform_final(final)
            # 변형된 자모를 다시 합성
            transformed_text.append(compose_hangul([new_initial, new_medial, new_final]))
        else:
            transformed_text.append(char)  # 한글이 아니면 그대로 추가

    return ''.join(transformed_text)

In [ ]:
# train['transformed'] 적용 시 예외 처리 추가
train['transformed'] = train['output'].apply(lambda x: transform_text(x))

# 텍스트 확인
train.head()

,ID,input,output,transformed
0,TRAIN_00000,별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈...,별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자...,범 홰 궤툐 와걁타. 외 솨럈듦의 퍾 1퀘릌 쥬닀츼 껐여봀 사럊의료쒀 맏료 셫멷햐챠...
1,TRAIN_00001,잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ,잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ,촹먓 차교 꺙 턔 춋눼오. 촧욤토 쥹 ㅋ
2,TRAIN_00002,절테 간면 않 된는 굣 멥몫,절대 가면 안 되는 곳 메모,졋뙤 갸몊 얘 톄늢 꼰 메모
3,TRAIN_00003,야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥류만 묵...,아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...,와... 가꼅 촐쿄 뷰도 폁 뚥러쒀 씌었하쥐맕 돥푀 넃쇄 뮈처뻐림. 샤꼐 화루맔 뮺...
4,TRAIN_00004,집윈 축쳐눌료 딴너왓눈뎁 카셩뷔 좋곱 칼쿰한네올. 쩌럼한뒈 뮬콰 욺료토 잊쿄 빻토 ...,지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...,찌읥 쭈첚의로 탸너웼늬댸 과섳뷔 쫌꼬 꺚끙화내요. 쭤렆홤퇘 뮶괴 읮료도 읫코 뺥됴 ...


In [ ]:
def decompose_hangul_2(char):
    """한글 문자를 초성, 중성, 종성으로 분해"""
    code = ord(char) - 0xAC00
    if code < 0 or code > 11171:
        return 0, 0, 0

    final = code % 28
    medial = (code // 28) % 21
    initial = code // 28 // 21
    return initial, medial, final

def compose_hangul_2(initial, medial, final):
    """초성, 중성, 종성 인덱스로 한글 음절 조합"""
    code = 0xAC00 + (initial * 21 + medial) * 28 + final
    return chr(code)

# 종성 자음을 초성 자음으로 변환하는 매핑
FINAL_TO_INITIAL = {
    1: 0,     # ㄱ -> ㄱ
    2: 1,     # ㄲ -> ㄲ
    4: 2,     # ㄴ -> ㄴ
    7: 3,     # ㄷ -> ㄷ
    8: 5,     # ㄹ -> ㄹ
    16: 6,     # ㅁ -> ㅁ
    17: 7,     # ㅂ -> ㅂ
    19: 9,     # ㅅ -> ㅅ
    20: 10,    # ㅆ -> ㅆ
    21: 11,     # ㅇ -> ㅇ
    22: 12,     # ㅈ -> ㅈ
    23: 14,   # ㅊ -> ㅊ
    24: 15,   # ㅋ -> ㅋ
    25: 16,   # ㅌ -> ㅌ
    26: 17,   # ㅍ -> ㅍ
    27: 18,   # ㅎ -> ㅎ
}

# 복합 종성 분해 매핑
COMPLEX_FINALS = {
    3: (1, 9),    # ㄳ -> ㄱ,ㅅ
    5: (4, 12),    # ㄵ -> ㄴ,ㅈ
    6: (4, 18),   # ㄶ -> ㄴ,ㅎ
    9:  (8, 0),    # ㄺ -> ㄹ,ㄱ
    10: (8, 6),   # ㄻ -> ㄹ,ㅁ
    11: (8, 7),   # ㄼ -> ㄹ,ㅂ
    12: (8, 9),   # ㄽ -> ㄹ,ㅅ
    13: (8, 16),  # ㄾ -> ㄹ,ㅌ
    14: (8, 17),  # ㄿ -> ㄹ,ㅍ
    15: (8, 18),  # ㅀ -> ㄹ,ㅎ
    18: (17, 9)    # ㅄ -> ㅂ,ㅅ
}

def apply_liaison(text):
    if not text:
        return ""

    words = text.split()
    result = []

    for word in words:
        chars = list(word)
        i = 0
        while i < len(chars) - 1:
            curr_char = chars[i]
            next_char = chars[i + 1]

            try:
                curr_i, curr_m, curr_f = decompose_hangul_2(curr_char)
                next_i, next_m, next_f = decompose_hangul_2(next_char)

                # 현재 글자에 종성이 있고, 다음 글자가 'ㅇ'으로 시작할 때
                if curr_f > 0 and next_i == 11:  # 11은 'ㅇ'의 인덱스
                    if curr_f in COMPLEX_FINALS:
                        # 복합 종성 처리
                        first_final, second_final = COMPLEX_FINALS[curr_f]
                        chars[i] = compose_hangul_2(curr_i, curr_m, first_final)
                        # 두 번째 자음을 초성으로 변환할 때 매핑 사용
                        new_initial = FINAL_TO_INITIAL.get(second_final, second_final)
                        chars[i + 1] = compose_hangul_2(new_initial, next_m, next_f)
                    else:
                        # 단순 종성 처리
                        chars[i] = compose_hangul_2(curr_i, curr_m, 0)
                        # 종성을 초성으로 변환할 때 매핑 사용
                        new_initial = FINAL_TO_INITIAL.get(curr_f, curr_f)
                        chars[i + 1] = compose_hangul_2(new_initial, next_m, next_f)
            except:
                pass

            i += 1

        result.append(''.join(chars))

    return ' '.join(result)


In [ ]:
train['raw_liaison'] = train['output'].apply(lambda x: apply_liaison(x))
# 텍스트 확인
train.head(10)

,ID,input,output,transformed,raw_liaison
0,TRAIN_00000,별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈...,별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자...,범 홰 궤툐 와걁타. 외 솨럈듦의 퍾 1퀘릌 쥬닀츼 껐여봀 사럊의료쒀 맏료 셫멷햐챠...,별 한 개도 아깝다. 왜 사람드리 별 1개를 주는지 겨꺼본 사라므로서 말로 설명하자...
1,TRAIN_00001,잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ,잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ,촹먓 차교 꺙 턔 춋눼오. 촧욤토 쥹 ㅋ,잠만 자고 갈 때 좋네요. 자못도 줌 ㅋ
2,TRAIN_00002,절테 간면 않 된는 굣 멥몫,절대 가면 안 되는 곳 메모,졋뙤 갸몊 얘 톄늢 꼰 메모,절대 가면 안 되는 곳 메모
3,TRAIN_00003,야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥류만 묵...,아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...,와... 가꼅 촐쿄 뷰도 폁 뚥러쒀 씌었하쥐맕 돥푀 넃쇄 뮈처뻐림. 샤꼐 화루맔 뮺...,아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...
4,TRAIN_00004,집윈 축쳐눌료 딴너왓눈뎁 카셩뷔 좋곱 칼쿰한네올. 쩌럼한뒈 뮬콰 욺료토 잊쿄 빻토 ...,지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...,찌읥 쭈첚의로 탸너웼늬댸 과섳뷔 쫌꼬 꺚끙화내요. 쭤렆홤퇘 뮶괴 읮료도 읫코 뺥됴 ...,지인 추처느로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...
5,TRAIN_00005,펀냔휜 잘 쉭곶 왔쑵닝따. 준윙에 맏쥡됴 만학썩 좋흖 겼 갇따용.,편안히 잘 쉬고 왔습니다. 주위에 맛집도 많아서 좋은 것 같아요.,폈얎희 쫴 싀코 웸씘뉘돠. 쮸의왜 맢찦도 맑아쒀 촗읭 컿 캊와요.,펴난히 잘 쉬고 왔습니다. 주위에 맛집도 만하서 조흔 것 가타요.
6,TRAIN_00006,"쓰윔튿룸위 깝써칙갖 있네 ㅋ 꼭굽 췸규웨 낑펙투, 합뻔퓨 옥쪽, 캬뻬 깥은 띰 떼잎...","스위트룸의 값어치가 있네 ㅋ 고급 침구에 킹베드, 하버뷰 욕조, 카페 같은 티 테이...","싀이드룬위 갎워쯰가 윝놰 ㅋ 고긣 췽뀨예 퀶패틔, 하뻐푸 욧쪼, 꽈패 괈은 띠 떄이...","스위트루믜 갑서치가 있네 ㅋ 고급 침구에 킹베드, 하버뷰 욕조, 카페 가튼 티 테이..."
7,TRAIN_00007,쨉빵뮨 의싸 쩐허 없쓺. 씬랏슬톄잎뾰댜 떠 짝음. 홈뗄 줄찾짱 15땜만 캉눙. 혹텔...,재방문 의사 전혀 없음. 신라스테이보다 더 작음. 호텔 주차장 15대만 가능. 호텔...,쩨뽳묻 으싸 쭯혀 웛읫. 쉷롸씌퉤이뵤톼 뚸 쟓의. 호뛚 쥬차좞 15댸먚 콰늪. 효델...,재방문 의사 전혀 업슴. 신라스테이보다 더 자금. 호텔 주차장 15대만 가능. 호텔...
8,TRAIN_00008,념묵 멎쥑교 꽁귀 좋습니닸. 췐곡윕뉘댜!,너무 멋지고 공기 좋습니다. 최고입니다!,녀뮤 뭡쥐꼬 콚퀴 춒슨뉘따. 체꼬윏늬톼!,너무 멋지고 공기 좋습니다. 최고입니다!
9,TRAIN_00009,"짱졈: 쩡켤함, 츄챠 씨셜 죠흠, 쉼섦 좋음. 단젊: 줏짢 츌짜할 떼 뷸뻔함, ...","장점: 청결함, 주차 시설 좋음, 시설 좋음. 단점: 주차 출차할 때 불편함, ...","짶쩟: 쭮꼂홮, 쮸짜 싀셚 좆이, 씌쉅 좇읰. 댣졆: 츄차 죿촤핛 데 푿퍾홥, ...","장점: 청결함, 주차 시설 조흠, 시설 조흠. 단점: 주차 출차할 때 불편함, 룸 ..."


In [ ]:
train['liaison_trans'] = train['raw_liaison'].apply(lambda x: transform_text(x))
train.head()

,ID,input,output,transformed,raw_liaison,liaison_trans
0,TRAIN_00000,별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈...,별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자...,범 홰 궤툐 와걁타. 외 솨럈듦의 퍾 1퀘릌 쥬닀츼 껐여봀 사럊의료쒀 맏료 셫멷햐챠...,별 한 개도 아깝다. 왜 사람드리 별 1개를 주는지 겨꺼본 사라므로서 말로 설명하자...,뼣 홮 케뚀 와콳탸. 왜 쏴랃트리 뻧 1꼐릛 추늹취 켜켜푭 솨랴믜료숴 뫌로 쒓멊하짜...
1,TRAIN_00001,잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ,잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ,촹먓 차교 꺙 턔 춋눼오. 촧욤토 쥹 ㅋ,잠만 자고 갈 때 좋네요. 자못도 줌 ㅋ,좑뫕 좌코 캈 떼 쭁냬요. 차묨또 충 ㅋ
2,TRAIN_00002,절테 간면 않 된는 굣 멥몫,절대 가면 안 되는 곳 메모,졋뙤 갸몊 얘 톄늢 꼰 메모,절대 가면 안 되는 곳 메모,쩍태 갸멹 앑 테늴 곧 먜모
3,TRAIN_00003,야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥류만 묵...,아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...,와... 가꼅 촐쿄 뷰도 폁 뚥러쒀 씌었하쥐맕 돥푀 넃쇄 뮈처뻐림. 샤꼐 화루맔 뮺...,아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...,와... 콰겆 촉교 푸또 쀁 둥려쒀 씨에햐취먋 댞뾔 녪세 믜쳐풔릿. 사쾌 화루맣 뮳...
4,TRAIN_00004,집윈 축쳐눌료 딴너왓눈뎁 카셩뷔 좋곱 칼쿰한네올. 쩌럼한뒈 뮬콰 욺료토 잊쿄 빻토 ...,지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...,찌읥 쭈첚의로 탸너웼늬댸 과섳뷔 쫌꼬 꺚끙화내요. 쭤렆홤퇘 뮶괴 읮료도 읫코 뺥됴 ...,지인 추처느로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...,즤읣 주쩌늬료 댜녀왝늹떄 카쎡뷔 졲꼬 캽긠화내오. 쭤렴홗테 뭄퀘 의료또 윟꾜 뽬뚀 ...


gemma

In [ ]:
train["prompt_input"] = "난독화된 문장을 복원하세요: " + train["input"]
test["prompt_input"] = "난독화된 문장을 복원하세요: " + test["input"]

# 데이터 분할
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(train, test_size=0.2, random_state=42)

from huggingface_hub import login

login("API_KEY")  # hugging face에서 api 발급 받음

# Tokenizer & Model 로드
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-

In [ ]:
import os
import pandas as pd
import torch
import random
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AdamW,
    get_scheduler,
)
from accelerate import Accelerator
from tqdm import tqdm
from sklearn.model_selection import train_test_split

###################################
# 1) 데이터 불러오기
###################################
train_path = "./train.csv"
test_path = "./test.csv"
train = pd.read_csv(train_path, encoding='utf-8-sig')
test = pd.read_csv(test_path, encoding='utf-8-sig')

###################################
# 2) 한글 증강/전처리 함수들
###################################
def decompose_hangul(char):
    base_code = ord(char) - 0xAC00
    initial_idx = base_code // 588
    medial_idx = (base_code % 588) // 28
    final_idx = base_code % 28
    initial = chr(initial_idx + 0x1100)  # 초성
    medial = chr(medial_idx + 0x1161)    # 중성
    final = chr(final_idx + 0x11A7) if final_idx > 0 else ''
    return initial, medial, final

initial_transform = {
    chr(0x1100): [chr(0x1101), chr(0x110F), chr(0x1100)],
    # ...
}
medial_transform = {
    chr(0x1161): [chr(0x1163), chr(0x116A), chr(0x1161)],
    # ...
}
final_consonants = [chr(0x11A8 + i) for i in range(28)]  # 종성 코드 (0x11A8 ~ 0x11C2)

def transform_initial(initial):
    if initial in initial_transform:
        return random.choice(initial_transform[initial])
    return initial

def transform_medial(medial):
    if medial in medial_transform:
        return random.choice(medial_transform[medial])
    return medial

def transform_final(final):
    if final:
        return random.choice(final_consonants)
    return ''

def compose_hangul(decomposed):
    HANGUL_BASE = 0xAC00
    INITIAL_BASE = 0x1100
    MEDIAL_BASE = 0x1161
    FINAL_BASE = 0x11A7
    initial, medial, final = decomposed
    initial_idx = ord(initial) - INITIAL_BASE
    medial_idx = ord(medial) - MEDIAL_BASE
    final_idx = ord(final) - FINAL_BASE if final else 0
    return chr(HANGUL_BASE + (initial_idx * 588) + (medial_idx * 28) + final_idx)

def transform_text(text):
    transformed_text = []
    for char in text:
        if '가' <= char <= '힣':
            i, m, f = decompose_hangul(char)
            new_i = transform_initial(i)
            new_m = transform_medial(m)
            new_f = transform_final(f)
            transformed_text.append(compose_hangul([new_i, new_m, new_f]))
        else:
            transformed_text.append(char)
    return ''.join(transformed_text)

# 연결음 처리 예시 함수
def decompose_hangul_2(char):
    code = ord(char) - 0xAC00
    if code < 0 or code > 11171:
        return 0, 0, 0
    final = code % 28
    medial = (code // 28) % 21
    initial = code // 28 // 21
    return initial, medial, final

def compose_hangul_2(initial, medial, final):
    return chr(0xAC00 + (initial * 21 + medial) * 28 + final)

FINAL_TO_INITIAL = {
    1: 0, 2: 1, 4: 2, 7: 3, 8: 5, 16: 6, 17: 7,
    19: 9, 20: 10, 21: 11, 22: 12, 23: 14,
    24: 15, 25: 16, 26: 17, 27: 18
}
COMPLEX_FINALS = {
    3: (1, 9), 5: (4, 12), 6: (4, 18), 9: (8, 0), 10: (8, 6),
    11: (8, 7), 12: (8, 9), 13: (8, 16), 14: (8, 17),
    15: (8, 18), 18: (17, 9)
}

def apply_liaison(text):
    if not text:
        return ""
    words = text.split()
    result = []
    for word in words:
        chars = list(word)
        i = 0
        while i < len(chars) - 1:
            curr_char = chars[i]
            next_char = chars[i + 1]
            try:
                curr_i, curr_m, curr_f = decompose_hangul_2(curr_char)
                next_i, next_m, next_f = decompose_hangul_2(next_char)
                # 종성 있고, 다음 글자 초성=='ㅇ' 일 때
                if curr_f > 0 and next_i == 11:
                    if curr_f in COMPLEX_FINALS:
                        f1, f2 = COMPLEX_FINALS[curr_f]
                        chars[i] = compose_hangul_2(curr_i, curr_m, f1)
                        new_init = FINAL_TO_INITIAL.get(f2, f2)
                        chars[i + 1] = compose_hangul_2(new_init, next_m, next_f)
                    else:
                        chars[i] = compose_hangul_2(curr_i, curr_m, 0)
                        new_init = FINAL_TO_INITIAL.get(curr_f, curr_f)
                        chars[i + 1] = compose_hangul_2(new_init, next_m, next_f)
            except:
                pass
            i += 1
        result.append(''.join(chars))
    return ' '.join(result)

# 전처리 컬럼 생성
train["raw_liaison"] = train["output"].apply(lambda x: apply_liaison(x))
train["liaison_trans"] = train["raw_liaison"].apply(lambda x: transform_text(x))

train["prompt_input"] = "난독화된 문장을 복원하세요: " + train["input"]
test["prompt_input"] = "난독화된 문장을 복원하세요: " + test["input"]

###################################
# 3) 데이터셋 분리
###################################
train_data, val_data = train_test_split(train, test_size=0.2, random_state=42)

###################################
# 4) 토크나이저/모델 불러오기
###################################
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")

###################################
# 5) Dataset 정의
###################################
class ReviewDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        input_text = row["prompt_input"]
        target_text = row["output"]

        input_enc = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        target_enc = self.tokenizer(
            target_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": input_enc["input_ids"].squeeze(),
            "attention_mask": input_enc["attention_mask"].squeeze(),
            "labels": target_enc["input_ids"].squeeze(),
        }

train_dataset = ReviewDataset(train_data, tokenizer)
val_dataset = ReviewDataset(val_data, tokenizer)

###################################
# 6) Accelerate 설정
###################################
accelerator = Accelerator(mixed_precision="fp16")

###################################
# 7) DataLoader, Optimizer, Scheduler 정의
###################################
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, pin_memory=True)

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 10
num_training_steps = len(train_dataloader) * num_epochs
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

###################################
# (추가) 체크포인트 불러오기(모델만)
###################################
resume_checkpoint = "best_model.pth"  # 불러올 체크포인트 경로
if os.path.exists(resume_checkpoint):
    print(f"[INFO] Found checkpoint at {resume_checkpoint}. Loading...")
    # 아직 Accelerator.prepare 전이므로 그냥 model.load_state_dict(...) 가능
    state_dict = torch.load(resume_checkpoint, map_location="cpu")
    model.load_state_dict(state_dict)
    print("[INFO] Checkpoint loaded!")

###################################
# 8) Accelerator 준비
###################################
model, optimizer, train_dataloader, val_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, val_dataloader, lr_scheduler
)

if hasattr(model.config, "use_cache"):
    model.config.use_cache = False
model.gradient_checkpointing_enable()

###################################
# 9) 학습 루프 (체크포인트 저장/로드 포함)
###################################
best_val_loss = float('inf')
patience = 3
early_stop_counter = 0

for epoch in range(num_epochs):
    accelerator.print(f"\n=== Epoch {epoch + 1}/{num_epochs} ===")

    # Training
    model.train()
    total_train_loss = 0.0

    for step, batch in enumerate(tqdm(train_dataloader, desc="Training", disable=not accelerator.is_local_main_process)):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_dataloader)
    accelerator.print(f"Train Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Validation", disable=not accelerator.is_local_main_process):
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            labels = batch["labels"]

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            total_val_loss += outputs.loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    accelerator.print(f"Validation Loss: {avg_val_loss:.4f}")

    # Early Stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stop_counter = 0
        if accelerator.is_main_process:
            accelerator.print("Best model saved.")
            # 모델 파라미터만 저장
            # (옵티마이저/스케줄러 상태까지 저장하려면 accelerator.save_state() 사용)
            accelerator.save(
                accelerator.unwrap_model(model).state_dict(),
                "best_model.pth"
            )
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            accelerator.print("Early stopping triggered.")
            break

accelerator.print("Training Complete!")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using device: cuda

=== Epoch 1/10 ===


Training:   0%|                                                                                | 0/2253 [00:00<?, ?it/s]/tmp/ipykernel_4103380/2143228523.py:69: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=torch.float16):
Training:   0%|                                                                        | 1/2253 [00:00<08:23,  4.47it/s]/home/aix23902/anaconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
Training:   0%|▎                                                 

OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 47.54 GiB of which 122.75 MiB is free. Including non-PyTorch memory, this process has 47.41 GiB memory in use. Of the allocated memory 47.07 GiB is allocated by PyTorch, and 24.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)